# Importing Gaia data to Firefly
*Author: Aaron Geller*

*Building off of the tutorial here: http://gea.esac.esa.int/archive-help/index.html *

*I believe the main table is gaiadr2.gaia_source, and [here](http://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html) is the description of the columns*

In [10]:
%load_ext autoreload
%autoreload 2
from FIREreader import FIREreader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
import astropy.units as u
import astropy.coordinates as coord
from astropy.coordinates.sky_coordinate import SkyCoord
from astroquery.gaia import Gaia
import numpy as np

*Do the following to load and look at the available Gaia table names:*

In [ ]:
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print (table.get_qualified_name())

In [ ]:
gaiadr2_table = Gaia.load_table('gaiadr2.gaia_source')
for column in (gaiadr2_table.get_columns()):
    print(column.get_name())

*Next, we retrieve all the available data *

*Note: The query to the archive is with ADQL (Astronomical Data Query Language). *

In [214]:
#there aren't any RVs
#cmd = "SELECT TOP 100 ra, dec, parallax, pmra, pmdec, radial_velocity FROM gaiadr2.gaia_source"

cmd = "SELECT TOP 2000 ra, dec, parallax, pmra, pmdec FROM gaiadr2.gaia_source \
WHERE phot_g_mean_mag<=10"

#synchronous commands are OK for jobs with < 2000 output rows
job = Gaia.launch_job(cmd, dump_to_file=False) 

#asynchronous commands are giving me an error!
# cmd = "SELECT * ra, dec, parallax, pmra, pmdec FROM gaiadr2.gaia_source \
# WHERE phot_g_mean_mag<=10"
#job = Gaia.launch_job_async(cmd, dump_to_file=False)

print (job)

<Table masked=True length=2000>
  name    dtype     unit    format                description                 n_bad
-------- ------- ---------- ------ ------------------------------------------ -----
      ra float64        deg {!r:>}                            Right ascension     0
     dec float64        deg {!r:>}                                Declination     0
parallax float64        mas {!r:>}                                   Parallax    41
    pmra float64 mas.yr**-1 {!r:>} Proper motion in right ascension direction    41
   pmdec float64 mas.yr**-1 {!r:>}     Proper motion in declination direction    41
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20180501154814.xml.gz
Results: None


*Inspect the output table and number of rows:*

In [215]:
r = job.get_results()
print(len(r))
print(r)

2000
        ra                 dec         ...        pmdec       
       deg                 deg         ...      mas.yr**-1    
------------------ ------------------- ... -------------------
  326.046610958014   9.875017848222623 ...                  --
 89.88182329162628   44.94743204476693 ...                  --
14.177441204227053   60.71672247798855 ...                  --
 274.4061143187362  -36.76240607839242 ... -164.92287295610117
221.24649351883244   27.07431392249596 ...                  --
109.28559761663782 -37.097446909729776 ...    2.28343468295344
311.55470104855004   33.97165785643453 ...   308.7866768543131
 296.5649839440163  10.613253300439279 ... -0.4016424904745284
236.06755715935202   6.425826685796823 ...  46.915636208302175
226.01721301839848 -25.282153195983703 ...  -38.54051987415152
               ...                 ... ...                 ...
170.83972114695513 -18.780112390751903 ...  -20.23587202300952
133.91527190537553   27.92734208039435 ...  -32.80

*Convert these ra, dec, parallax coordinates to 3D cartesian*

In [216]:

#it is odd that some of the parallaxes are negative!
print(len(np.where(r['parallax'] < 0)[0]))
dist = np.abs((r['parallax']).to(u.parsec, equivalencies=u.parallax()))
astroC = SkyCoord(ra=r['ra'], dec=r['dec'], distance=dist).cartesian #transform_to(coord.Galactocentric)

#this would account for velocities also
# pmra = np.array([x for x in r['pmra']]) * u.mas/u.yr
# pmdec = np.array([x for x in r['pmdec']]) * u.mas/u.yr
# #there aren't any RVs
# #rv = np.array([x for x in r['radial_velocity']]) * u.km/u.s 
# rv = np.zeros(len(pmra)) * u.km/u.s
# astroC = coord.ICRS(ra=r['ra'], dec=r['dec'], distance=dist, 
#                     pm_ra_cosdec=pmra, pm_dec=pmdec, radial_velocity=rv).transform_to(coord.Galactocentric)

4


/Users/ageller/anaconda/envs/py36/lib/python3.6/site-packages/astropy/table/column.py:954: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)


*Now put these in the format expected by FIREreader*

In [217]:
coords = []
vel = []
for i,c in enumerate(astroC):
    if (np.isfinite(c.x)):
        if (len(coords) == 0):
            coords = np.array([[c.x.to(u.pc).value, c.y.to(u.pc).value ,c.z.to(u.pc).value]])
#             vel = np.array([[c.v_x.to(u.km/u.s).value, c.v_y.to(u.km/u.s).value ,c.v_z.to(u.km/u.s).value]])
        else:
            coords = np.append(coords, \
                   np.array([[c.x.to(u.pc).value, c.y.to(u.pc).value ,c.z.to(u.pc).value]]), axis=0)
#             vel = np.append(vel, \
#                     np.array([[c.v_x.to(u.km/u.s).value, c.v_y.to(u.km/u.s).value ,c.v_z.to(u.km/u.s).value]]), axis=0)

            
print(coords[0:10])
# print(vel[0:10])

[[    3.1315411    -40.64130966   -30.45201359]
 [  -69.37069534   198.25135221  -158.83565473]
 [   12.74090792   -14.37330244    12.94170654]
 [  160.33843567  -320.67781207    67.18262268]
 [  -14.08974221   -20.94212353     2.84272035]
 [  -79.24365749   -82.10856      -53.89665179]
 [  108.7320149  -1183.62180629  -681.49846896]
 [   26.02758967    44.19954352   -12.32302709]
 [  -31.68304766  -207.65290788  -305.97646536]
 [  -69.26527358    31.44668857    67.29982114]]


*Create the Firefly data files using FIREreader*

In [218]:
pname = 'Gaia'

reader = FIREreader()

#set all of these manually
reader.returnParts = [pname] 
reader.names = {pname:pname}
reader.dataDir = pname
reader.JSONfname = pname+'Data'

#create a space for the data in partsDict
reader.partsDict[pname] = dict()

#define the defaults; this must be run first if you want to change the defaults below
reader.defineDefaults()
reader.defineFilterKeys()

#update a few of the options
#reader.options['sizeMult'][pname] = 0.1
reader.options['color'][pname] = [1., 1., 0., 1.]
reader.options['center'] = [0,0,0]

reader.addtodict(reader.partsDict,None,'Gaia','Coordinates',0,0,vals=coords)


reader.shuffle_dict()
reader.swap_dict_names()
reader.createJSON()

shuffling ... 
writing JSON files ...
Gaia
Gaia/GaiaDataOptions.json
